**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from class_utils import error_histogram

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("sigmoid_regression_data.csv"), directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Decision Trees for Regression

Decision trees can be applied to both: classification and regression. We will now show a very straight-forward example of how to apply them to regression. The dataset that we will be using is 2-dimensional, which will enable us to plot it easily and to examine the regression curve visually. The code to load and preprocess the data follows in the next cell and is hidden for the sake of brevity.



In [ ]:
#@title -- Loading and Preprocessing the Data: X_train, Y_train, X_test, Y_test -- { display-mode: "form" }
df = pd.read_csv("data/sigmoid_regression_data.csv")

kbins = KBinsDiscretizer(6, encode='ordinal')
y_stratify = kbins.fit_transform(df[['y']])

df_train, df_test = train_test_split(
    df, stratify=y_stratify, test_size=0.3, random_state=4)

plt.scatter(df_train['x'], df_train['y'], marker='x', label="training data")
plt.scatter(df_test['x'], df_test['y'], c='r', label="testing data")
plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()

categorical_inputs = []
numeric_inputs = ['x']
output = 'y'

input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values

### Creating and Training the Regressor

To create the regressor, we merely need to instantiate class `DecisionTreeRegressor`. We will be using the default parameters so we do not need to specify anything at this stage. The training will be done using the standard `fit` interface.



In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, Y_train)

### Testing

The testing will be done by measuring the mean square error (MSE) and the mean absolute error (MAE) of our model on the test set. We will also display the error histogram to get a better idea of how the errors are distributed and what their scale is w.r.t. the original data.



In [ ]:
#@title -- Testing -- { display-mode: "form" }
y_test = model.predict(X_test)

# we compute and display the MSE and the MAE
mse = mean_squared_error(Y_test, y_test)
print("MSE = {}".format(mse))

mae = mean_absolute_error(Y_test, y_test)
print("MAE = {}".format(mae))

# we display the error histogram
plt.figure(figsize=(8, 6))
error_histogram(Y_test, y_test, Y_fit_scaling=Y_train)

The results should quite good: the errors should be negligible compared to the scale of the data.

### Visualizing the Regression Curve

When we visualize the regression curve, we will notice that it is not smooth, but rather piece-wise constant. This is because each leaf of the tree corresponds to a certain area of the input space and it predicts a single constant value. Consequently the input area will be divided into discrete areas with various constant outputs.



In [ ]:
#@title -- Regression Curve vs. Data -- { display-mode: "form" }
x_min = min(np.min(X_train), np.min(X_test))
x_max = max(np.max(X_train), np.max(X_test))

xx = np.linspace(x_min, x_max, 250).reshape((-1, 1)).astype(np.float32)
yy = model.predict(xx)

plt.scatter(X_train, Y_train, marker='x', label="training data")
plt.scatter(X_test, Y_test, c='r', label="testing data")

plt.plot(xx, yy, label="regression curve", c='k')

plt.xlabel('x')
plt.ylabel('y')
plt.grid(ls='--')
plt.legend()